In [1]:
import numpy as np
import _pickle as pickle
import glob
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from music21 import note, chord, instrument, converter
from keras.layers import LSTM, Dense, Dropout, Activation, Input
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint

/usr/local/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Set to true for debugging
debug = False

In [8]:
# Main function to call all subfunctions in the notebook.
def train_network():
    # get data and convert it to notes
    notes, n_vocab = get_notes(quick=True)
    # prepare data
    mapped_notes, NetworkInput, NetworkOutut = prepare_data(notes, n_vocab)
    # get model
    model = get_model(NetworkInput.shape[1:], n_vocab)
    # train
    train(model, NetworkInput, NetworkOutut)
#--------------------------------------------------------------------
# uncomment after runing all cells.
train_network()

Notes are loaded properly using the saved pickle
Input shape =  (620, 100, 1) 
Output shape =  (620, 32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 1)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_____________________________________

Epoch 75/200
620/620 [==============================] - 129s 209ms/step - loss: 2.7580
Epoch 76/200
620/620 [==============================] - 132s 213ms/step - loss: 2.7519
Epoch 77/200
620/620 [==============================] - 132s 212ms/step - loss: 2.7533
Epoch 78/200
620/620 [==============================] - 132s 212ms/step - loss: 2.7615
Epoch 79/200
620/620 [==============================] - 130s 210ms/step - loss: 2.7455
Epoch 80/200
620/620 [==============================] - 130s 210ms/step - loss: 2.7482
Epoch 81/200
620/620 [==============================] - 131s 211ms/step - loss: 2.7485
Epoch 82/200
620/620 [==============================] - 131s 211ms/step - loss: 2.7780
Epoch 83/200
620/620 [==============================] - 132s 213ms/step - loss: 2.7755
Epoch 84/200
620/620 [==============================] - 130s 210ms/step - loss: 2.7679
Epoch 85/200
620/620 [==============================] - 131s 211ms/step - loss: 2.7415
Epoch 86/200
620/620 [=====================

620/620 [==============================] - 133s 214ms/step - loss: 2.7255
Epoch 169/200
620/620 [==============================] - 134s 216ms/step - loss: 2.7248
Epoch 170/200
620/620 [==============================] - 134s 216ms/step - loss: 2.7407
Epoch 171/200
620/620 [==============================] - 136s 219ms/step - loss: 2.7297
Epoch 172/200
620/620 [==============================] - 137s 222ms/step - loss: 2.7446
Epoch 173/200
620/620 [==============================] - 134s 216ms/step - loss: 2.7267
Epoch 174/200
620/620 [==============================] - 135s 217ms/step - loss: 2.7363
Epoch 175/200
620/620 [==============================] - 133s 214ms/step - loss: 2.7278
Epoch 176/200
620/620 [==============================] - 133s 215ms/step - loss: 2.7408
Epoch 177/200
620/620 [==============================] - 134s 217ms/step - loss: 2.7353
Epoch 178/200
620/620 [==============================] - 138s 223ms/step - loss: 2.7364
Epoch 179/200
620/620 [=======================

In [4]:
def get_notes(quick=False):
    ''' Read input midi files and convert them to notes
        Also quick refers to using the saved notes.pkl to retrieve notes instead of reading midi files.
    '''
    input_folder = './data/input_midi/'
    output_folder = './data/'
    notes = []
    
    if quick :
        try :
            with open(output_folder+'notes.pkl','rb') as f :
                notes = pickle.load(f)
            print('Notes are loaded properly using the saved pickle')
            return notes, len(set(notes))
        except :
            print('It\'s not possible to do it quick,\n reading midi files....')
            return get_notes()
    
    for file in glob.glob(input_folder+'*.mid'):
        midi = converter.parse(file)
        notes_to_parse  = []
        parts = instrument.partitionByInstrument(midi)
        
        if parts :
            notes_to_parse = parts.parts[0].recurse()
        else :
            notes_to_parse = parts.flat_notes
        
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(e) for e in c.pitches))
            else : # it shouldn't reach here.
                pass
        with open(output_folder+'notes.pkl', 'wb') as f :
            pickle.dump(notes, f)
        print('data loaded properly and saved to disk as notes.pkl.')
        return notes, len(set(notes))
#--------------------------------------------------------
if debug : notes, n_vocab = get_notes(quick=True)

In [5]:
def prepare_data(notes, n_vocab):
    ''' create input sequences and output notes '''
    sequence_length = 100
    NetworkInput = []
    NetworkOutput = []
    # create a mapping to the notes
    mapper = LabelEncoder()
    mapped_notes = mapper.fit_transform(notes)

    for i in range(len(notes)-sequence_length):
        in_seq = mapped_notes[i : i+sequence_length]
        out_note = mapped_notes[i+sequence_length]
        NetworkInput.append(in_seq)
        NetworkOutput.append(out_note)
    
    n_patterns = len(NetworkOutput)
    
    NetworkInput = np.reshape(NetworkInput, (n_patterns, sequence_length, 1))
    NetworkInput = NetworkInput / float(n_vocab)
    
    NetworkOutput = np.reshape(NetworkOutput, (-1,1))
    hotencoder = OneHotEncoder(sparse=False)
    _ = hotencoder.fit(mapped_notes.reshape(-1,1))
    NetworkOutput = hotencoder.transform(NetworkOutput)
    
    # save the mapper and hotencoder to disk for prediction.
    #with open('./data/mapper.pkl','wb') as f:
    #    pickle.dump(mapper, f)
    #with open('./data/hotencoder.pkl','wb') as f:
    #    pickle.dump(hotencoder, f)
    
    print('Input shape = ',NetworkInput.shape, '\nOutput shape = ', NetworkOutput.shape)
    return mapped_notes, NetworkInput, NetworkOutput
#---------------------------------------
if debug : mapped_notes, NetworkInput, NetworkOutput = prepare_data(notes, n_vocab)

In [6]:
def get_model(input_shape, n_vocab):
    
    X_input = Input(input_shape)
    X = LSTM(512, activation='tanh', return_sequences=True)(X_input)
    X = Dropout(.3)(X)
    X = LSTM(512, activation='tanh', return_sequences=True)(X)
    X = Dropout(.3)(X)
    X = LSTM(512, activation='tanh', return_sequences=False)(X)
    X = Dropout(.3)(X)
    X = Dense(256, activation='tanh')(X)
    X = Dropout(.3)(X)
    X = Dense(n_vocab, activation='softmax')(X)
    
    model = Model(X_input, X)
    model.compile(loss='categorical_crossentropy', optimizer='Adam')
    
    model.summary()
    return model
#----------------------------------------------
if debug : model = get_model(NetworkInput.shape[1:], n_vocab)

In [7]:
def train(model, NetworkInput, NetworkOutput):
    filepath = './checkpoints/0/ckpt-{epoch:02d}-{loss:.4f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
    callbacks=[checkpoint]
    
    model.fit(NetworkInput, NetworkOutput, epochs=1000, batch_size=16, callbacks=callbacks)
    return model
#-----------------------------------
if debug : trained_model = train(model, NetworkInput, NetworkOutput)